In [4]:
##do_runcode
##%overwritefile
##%file:src/do_go_runcode.py
##%noruncode
    def do_runcode(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=return_code
        fil_ename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        ##代码运行前
        ################# repl mode run code files
        #FIXME:
        if len(self.addkey2dict(magics,'execfile'))>0:
            fil_ename=magics['execfile']
        if magics['_st']['runmode']=='repl':
            self._start_replprg(fil_ename,magics['_st']['args'],magics)

            return_code=self.replwrapper.child.status
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
            return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        ############################################

    ############################################
        p=None
        #################dynamically load and execute code
        #FIXME:
        
        if len(magics['dlrun'])>0:
            p = self.create_jupyter_subprocess([self.master_path, fil_ename] + magics['_st']['args'],env=self.addkey2dict(magics,'env'),magics=magics)
        #################
        else:
            p = self.create_jupyter_subprocess([fil_ename] + magics['_st']['args'],env=self.addkey2dict(magics,'env'),magics=magics)
        self.subprocess=p
        self.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
        # if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        if len(self.addkey2dict(magics,'showpid'))>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        p.wait_end(magics)
        return_code=p.returncode
        ##代码运行结束
        # now remove the files we have just created
        # if(os.path.exists(source_file.name)):
            # os.remove(source_file.name)
        # if(os.path.exists(binary_filename)):
            # os.remove(binary_filename)

        # if p.returncode != 0:
            # self._write_to_stderr("[C kernel] Executable exited with code {}".format(p.returncode))
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\src/do_go_runcode.py created successfully


In [5]:
##do_compile_code
##%overwritefile
##%file:src/do_go_compilecode.py
##%noruncode
    def do_compile_code(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=fil_ename
        sourcefilename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''

        returncode,binary_filename=self._exec_goc_(fil_ename,magics)
        fil_ename=binary_filename
        return_code=returncode
        
        if returncode!=0:return  True,retinfo, code,fil_ename,retstr

        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\src/do_go_compilecode.py created successfully


In [6]:
##do_create_codefile
##%overwritefile
##%file:src/do_go_create_codefile.py
##%noruncode
    def do_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=''
        sourcefilename=''
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''

        source_file=self.create_codetemp_file(magics,code,suffix='.go')
        sourcefilename=source_file.name 
        newsrcfilename=source_file.name
        fil_ename=newsrcfilename
        return_code=True

        return  bcancel_exec,self.get_retinfo(),magics, code,fil_ename,retstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\src/do_go_create_codefile.py created successfully


In [7]:
##do_preexecute
##%overwritefile
##%file:src/do_go_preexecute.py
##%noruncode
    def do_preexecute(self,code,magics, silent, store_history=True,
                user_expressions=None, allow_stdin=False):
        bcancel_exec=False
        retinfo=self.get_retinfo()
        ############# run gdb and send command begin
        if len(self.addkey2dict(magics,'rungdb'))>0:
            bcancel_exec=True
            retinfo= self.replgdb_sendcmd(code,silent, store_history,
                user_expressions, allow_stdin)
            return bcancel_exec,retinfo,magics, code
        ############# run gdb and send command
        #############send replcmd's command
        if magics['_st']['runmode']=='repl':
            if hasattr(self, 'replcmdwrapper'):
                if self.replcmdwrapper :
                    bcancel_exec=True
                    retinfo= self.repl_sendcmd(code, silent, store_history,
                        user_expressions, allow_stdin,magics)
                    return bcancel_exec,retinfo,magics, code
        # if len(self.addkey2dict(magics,'noruncode'))<1 :
        #     magics, code = self._add_main(magics, code)
        return bcancel_exec,retinfo,magics, code

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\src/do_go_preexecute.py created successfully


In [8]:
## new kernel.py
##%overwritefile
##%file:../../../jupyter-MyGo-kernel/jupyter_MyGo_kernel/kernel.py
###%file:go_kernel.py
##%noruncode
#
#   MyGo Jupyter Kernel 
#   generated by MyPython
#
##%include:../../src/head.py
##%include:../../src/common.py
##%include:../../src/Mykernel.py
class GoKernel(MyKernel):
    implementation = 'jupyter-MyGo-kernel'
    implementation_version = '1.0'
    language = 'Go'
    language_version = ''
    language_info = {'name': 'text/x-gosrc',
                     'mimetype': 'text/x-gosrc',
                     'file_extension': '.go'}
    runfiletype='exe'
    banner = "Go kernel.\n" \
             "Uses go, creates source code files and executables in temporary folder.\n"

    main_head = "#include <stdio.h>\n" \
            "#include <math.h>\n" \
            "int main(int argc, char* argv[], char** env){\n"

    main_foot = "\nreturn 0;\n}"



    def __init__(self, *args, **kwargs):
        super(GoKernel, self).__init__(*args, **kwargs)
        # MyKernel.__init__(self, cmd_or_spawn, orig_prompt,
        #                               prompt_change,extra_init_cmd=extra_init_cmd)
        self.kernelinfo='[MyGo Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        
#################
##%include:src/compile_with_goc.py
##%include:src/go_exec_goc_.py
##%include:src/c_start_gdb.py
##%include:src/c_do_replexecutegdb.py
##%include:src/c_replgdb_sendcmd.py

##%include:src/do_go_runcode.py
##%include:src/do_go_compilecode.py
##%include:src/do_go_create_codefile.py
##%include:src/do_go_preexecute.py



[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Go\../../../jupyter-MyGo-kernel/jupyter_MyGo_kernel/kernel.py created successfully
